In [ ]:
# -*- coding: utf-8 -*-
"""
train_card_policy_bc.py

Trains a card prediction policy using Behavioral Cloning (BC)
based on data parsed into an HDF5 file.

Assumes the HDF5 file contains 'state_bits' (bool, N x 929) and
'action_card_bits' (bool, N x 13) datasets, as generated by the parser.
Converts action_card_bits to action_index (int, N) during loading.

Uses the standard imitation.algorithms.bc.BC trainer.
Requires compatible versions of torch, gymnasium, stable-baselines3, and imitation.
"""

import h5py
import numpy as np
import gymnasium as gym
# stable_baselines3 is implicitly used by imitation for policy structure
import stable_baselines3 as sb3
# Import core BC algorithm and data types
from imitation.algorithms import bc
from imitation.data import types
# No need for TrainingMetrics or util imports here when using default loss_calculator

import logging
import os
import torch as th # Use th alias for torch conventions
import traceback # To print full error tracebacks
from typing import Tuple, Optional
from sklearn.model_selection import train_test_split # Corrected import


# --- Logging Configuration ---
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


# --- Configuration ---
HDF5_FILE_PATH = r'../../../Training_Data/jass.hdf5'  # <--- Path to the HDF5 file from your parser
POLICY_SAVE_PATH = 'jass_bc_card_policy_591' # Model save path

# --- Data Keys (Must match your parser's output HDF5 structure) ---
OBS_KEY = 'state_bits'          # N x 929 boolean array
ACTION_BITS_KEY = 'action_card_bits' # N x 13 boolean array (the played card's representation)

# --- Model Dimensions (Based on your parser) ---
INPUT_DIM = 929 # From your parser's STATE_BITS calculation
CARD_ACTION_DIM = 9 # 9 possible card positions in hand (index 0-8)

# --- Constants from Parser (Needed to interpret state_bits) ---
CARD_BITS = 13 # Bits used to represent a single card in the parser
NUM_CARDS_HISTORY = 32 # Number of history card slots in state (32 * 13 bits)
NUM_CARDS_TABLE = 3  # Number of table card slots in state (3 * 13 bits) - matches parser code, not comment
NUM_CARDS_HAND = 9   # Number of hand card slots in state (9 * 13 bits) - matches CARD_ACTION_DIM conceptually

# Calculate the starting index of the player's hand bits within the state_bits vector
# Order from parser: History, Table, Hand, Shown, Trump
HAND_START_BIT_INDEX = (NUM_CARDS_HISTORY * CARD_BITS) + (NUM_CARDS_TABLE * CARD_BITS)
# 32 * 13 = 416
# 3 * 13  = 39
# HAND_START_BIT_INDEX = 416 + 39 = 455. The hand bits are from index 455 up to 455 + (9*13) = 455 + 117 = 572.

# --- Training Hyperparameters ---
VALIDATION_SPLIT_SIZE = 0.15 # Use 15% of data for validation (optional for BC eval)
RANDOM_SEED = 42 # For reproducible train/test splits
BC_BATCH_SIZE = 32
BC_LEARNING_RATE = 3e-4
BC_N_EPOCHS = 10 # Number of passes over the training data

device = th.device("cpu")
logging.info(f"Using device: {device}")


# --- 1. Load Data Function (Kept Action Conversion) ---
def load_bc_data_from_hdf5(filepath: str) -> Tuple[Optional[np.ndarray], Optional[np.ndarray]]:
    """
    Loads observations and converts action card bits (N x 13 bool) to action indices (N x 1 int, 0-8)
    by finding the index of the played card within the hand representation in the state.
    Loads data from all groups in the HDF5 file.
    """
    all_obs = []
    all_action_indices = [] # Store the derived action indices (0-8)

    if not os.path.exists(filepath):
        logging.error(f"HDF5 file not found at: {filepath}")
        return None, None

    try:
        with h5py.File(filepath, 'r') as f:
            # Load data from a subset of groups for speed if needed, or all
            game_groups = list(f.keys()) # Load all groups
            #game_groups = list(f.keys())[:2000] # Example: Load only first 2000 games for faster testing
            logging.info(f"Found {len(game_groups)} game groups in HDF5 file for BC.")

            if not game_groups:
                logging.error("No game groups found in the HDF5 file.")
                return None, None

            for i, group_name in enumerate(game_groups):
                if (i + 1) % 100 == 0:
                    logging.info(f"Loading and processing group {i+1}/{len(game_groups)} for BC: {group_name}")
                try:
                    group = f[group_name]
                    # --- Check required keys ---
                    if OBS_KEY not in group:
                        logging.warning(f"BC Skipping group '{group_name}': Missing dataset '{OBS_KEY}'")
                        continue
                    # We now expect 'action_card_bits' from the parser
                    if ACTION_BITS_KEY not in group:
                        logging.error(f"BC CRITICAL: Skipping group '{group_name}': Missing dataset '{ACTION_BITS_KEY}'.")
                        logging.error(f"Ensure your parser saves the 13-bit card representation as '{ACTION_BITS_KEY}'.")
                        continue # Skip group if critical action data is missing

                    obs_group_data = group[OBS_KEY][:] # Shape (N, INPUT_DIM) bool -> N x 929
                    actions_bits_group_data = group[ACTION_BITS_KEY][:] # Shape (N, CARD_BITS) bool -> N x 13

                    # --- Basic Validation ---
                    if obs_group_data.shape[0] != actions_bits_group_data.shape[0]:
                        logging.warning(f"BC Skipping group '{group_name}': Data length mismatch ({obs_group_data.shape[0]} vs {actions_bits_group_data.shape[0]}).")
                        continue
                    if obs_group_data.shape[0] == 0:
                         logging.info(f"BC Skipping empty group '{group_name}'.")
                         continue
                    if obs_group_data.shape[1] != INPUT_DIM:
                        logging.warning(f"BC Skipping group '{group_name}': Observation dimension mismatch (Expected {INPUT_DIM}, Got {obs_group_data.shape[1]}).")
                        continue
                    # Validate the shape of the action bits data from the parser
                    if actions_bits_group_data.shape[1] != CARD_BITS:
                         logging.warning(f"BC Skipping group '{group_name}': Action bits dimension mismatch (Expected {CARD_BITS}, Got {actions_bits_group_data.shape[1]}).")
                         continue

                    # --- Convert Action Card Bits to Action Index (0-8) ---
                    # Iterate through each state-action pair in the group
                    group_derived_indices = []
                    # Convert boolean arrays to integers (0 or 1) for easier comparison/processing
                    # This conversion is safe as boolean True/False become 1/0
                    obs_group_int = obs_group_data.astype(np.int8)
                    actions_bits_group_int = actions_bits_group_data.astype(np.int8)

                    for j in range(obs_group_int.shape[0]):
                        state_vec_int = obs_group_int[j, :] # This sample's state (929 integers 0/1)
                        played_card_bits_int = actions_bits_group_int[j, :] # This sample's action card bits (13 integers 0/1)

                        # Extract the player's hand bits from the state vector
                        # Hand bits start at HAND_START_BIT_INDEX and are NUM_CARDS_HAND * CARD_BITS long
                        hand_bits_int = state_vec_int[HAND_START_BIT_INDEX : HAND_START_BIT_INDEX + NUM_CARDS_HAND * CARD_BITS]

                        # Find the index of the played_card_bits within the hand_bits
                        action_index = -1 # Default to not found
                        for card_idx_in_hand in range(NUM_CARDS_HAND):
                            # Extract the bits for the current card in the hand
                            start_idx = card_idx_in_hand * CARD_BITS
                            end_idx = start_idx + CARD_BITS
                            current_hand_card_bits_int = hand_bits_int[start_idx : end_idx]

                            # Compare the played card bits with the current hand card bits
                            # Use np.array_equal for a robust comparison of the 13-element vectors
                            if np.array_equal(played_card_bits_int, current_hand_card_bits_int):
                                action_index = card_idx_in_hand # Found the index!
                                break # Found the card in hand, its index is our action

                        if action_index == -1:
                            # This indicates a problem: the card recorded as played was not found in the state's hand
                            # (e.g., due to parsing errors, inconsistent state/action logging)
                            # We skip this sample as we cannot determine the correct action index (0-8)
                            # from the player's perspective at that state.
                            # You could log the bit sequences for debugging if this happens often.
                            # logging.warning(f"BC WARNING: Card played (bits: {played_card_bits_int.tolist()}) not found in hand (bits: {hand_bits_int.reshape(-1, CARD_BITS).tolist()}) for sample {j} in group '{group_name}'. Skipping this sample.")
                            continue # Skip to the next sample (j)

                        # If index was found (0-8), append it
                        group_derived_indices.append(action_index)

                    # Only append data from this group if we successfully derived at least one index
                    if group_derived_indices:
                        # Convert boolean observations to float32 for the model
                        all_obs.append(obs_group_data.astype(np.float32)) # Use original boolean data for obs -> float32
                        # Append the derived action indices for the samples that were not skipped
                        all_action_indices.append(np.array(group_derived_indices, dtype=np.int64)) # Ensure actions are int64
                    else:
                         logging.warning(f"BC: No valid action indices derived for group '{group_name}'. Skipping entire group.")


                except Exception as e:
                    logging.error(f"BC Error processing group '{group_name}': {e}")
                    traceback.print_exc() # Print traceback for detailed error
                    continue # Skip this group on error

            if not all_obs or not all_action_indices:
                logging.error("BC: No valid data loaded from any group.")
                return None, None

            # Concatenate data from all groups
            logging.info("BC: Concatenating data from all groups...")
            final_obs = np.concatenate(all_obs, axis=0)
            final_action_indices = np.concatenate(all_action_indices, axis=0)
            logging.info("BC: Concatenation complete.")
            logging.info(f"Final concatenated data shapes: obs={final_obs.shape}, actions={final_action_indices.shape}")


            # Final sanity check on action indices range (should be 0-8)
            # np.max and np.min will error on empty arrays, check size first
            if final_action_indices.size > 0:
                if np.max(final_action_indices) >= CARD_ACTION_DIM or np.min(final_action_indices) < 0:
                     logging.error(f"BC CRITICAL: Derived action indices out of range (0-{CARD_ACTION_DIM-1}). Max: {np.max(final_action_indices)}, Min: {np.min(final_action_indices)}")
                     # This indicates a major issue in the derivation logic or source data
                     # You might want to raise an error or filter these out explicitly if they exist
                     pass # Allow continuation for now, but investigate data if this warning appears often
            else:
                 logging.warning("No action indices were derived after loading and filtering data.")


            return final_obs, final_action_indices

    except Exception as e:
        logging.critical(f"BC: Failed to load data from HDF5 file '{filepath}': {e}")
        traceback.print_exc() # Print traceback for detailed error
        return None, None

observations, card_action_indices = load_bc_data_from_hdf5(HDF5_FILE_PATH)


/opt/conda/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:

# --- Main Execution ---
if __name__ == "__main__":
    logging.info("--- Starting Card Policy Training (BC) ---")

    # --- Load Data ---
    # We now expect the second return value to be the action indices (0-8)
    if observations is None or card_action_indices is None:
        logging.critical("Could not load data for BC training. Exiting.")
        exit()
    # Ensure observations and action indices have the same number of samples after loading and filtering
    if len(observations) != len(card_action_indices):
         logging.critical(f"Observation and action index counts mismatch after loading: {len(observations)} vs {len(card_action_indices)}. Exiting.")
         exit()
    if len(observations) == 0:
         logging.critical("No valid data samples loaded for BC training. Exiting.")
         exit()
    logging.info(f"Total valid data samples loaded for BC: {len(observations)}.")


    # --- Split Data (Optional for BC, but good practice) ---
    logging.info(f"Splitting data for BC (keeping {1-VALIDATION_SPLIT_SIZE:.0%} for training)...")
    # We primarily need the training split for BC demonstrations
    # Ensure both obs and action_indices are split together
    obs_train, obs_val, act_train_indices, act_val_indices = train_test_split(
        observations, card_action_indices, # Use the derived indices here
        test_size=VALIDATION_SPLIT_SIZE,
        random_state=RANDOM_SEED,
        shuffle=True,
        # stratify=card_action_indices # Optional: Stratify to ensure action distribution is similar in train/val splits - requires all indices to be valid (0-8). Uncomment if needed and all data is valid.
    )
    logging.info(f"BC Training samples: {len(obs_train)}, Validation samples: {len(obs_val)}")
    # Free up memory from the original full arrays and validation splits
    del observations, card_action_indices, obs_val, act_val_indices


    # --- Prepare Data & Spaces for Imitation Library ---
    # The Transitions object holds NumPy arrays. The batching logic in bc.train
    # will convert these to tensors and move them to the device using its internal
    # mechanism (e.g., safe_to_tensor).
    dummy_next_obs_array = np.zeros_like(obs_train)

    train_demonstrations = types.Transitions(
        obs=obs_train, # NumPy array
        acts=act_train_indices, # NumPy array of integer indices (0-8)
        infos=np.array([None] * len(obs_train), dtype=object), # Dummy infos (NumPy array)
        next_obs=dummy_next_obs_array, # Dummy next_obs (NumPy array)
        dones=np.array([False] * len(obs_train)), # Dummy dones (NumPy array)
    )
    # Free up memory from the arrays now held by the Transitions object
    del obs_train, act_train_indices


    observation_space = gym.spaces.Box(low=0, high=1, shape=(INPUT_DIM,), dtype=np.float32)
    action_space = gym.spaces.Discrete(CARD_ACTION_DIM) # Keep Discrete(9) for action index


    # --- Train Behavioral Cloning Model ---
    rng = np.random.default_rng(RANDOM_SEED)
    bc_trainer = bc.BC(
        observation_space=observation_space,
        action_space=action_space,
        demonstrations=train_demonstrations, # Pass the Transitions object (with NumPy arrays)
        batch_size=BC_BATCH_SIZE,
        optimizer_kwargs=dict(lr=BC_LEARNING_RATE),
        device=device, # Ensure the trainer/policy are created on the correct device
        rng=rng,
        # *** USING DEFAULT loss_calculator ***
        # If you encounter the device RuntimeError again, it means the default
        # loss_calculator in your imitation version has the bug.
        # In that case, you would need to use the minimalist custom loss calculator
        # code from the previous response, which should work if torch and numpy are available.
        # loss_calculator=CorrectedBehaviorCloningLossCalculator() # Uncomment if device bug persists with default
    )

    # Calculate n_batches based on the total number of training samples
    n_batches_per_epoch = len(train_demonstrations.obs) // BC_BATCH_SIZE
    if len(train_demonstrations.obs) % BC_BATCH_SIZE != 0:
        n_batches_per_epoch += 1 # Account for the last partial batch

    n_batches = n_batches_per_epoch * BC_N_EPOCHS
    # Ensure n_batches is at least 1 if there's data
    n_batches = max(1, n_batches) if len(train_demonstrations.obs) > 0 else 0


    logging.info(f"Starting BC training for {BC_N_EPOCHS} epochs (~{n_batches_per_epoch} batches per epoch) totaling {n_batches} batches...")
    if n_batches > 0:
        # Using n_batches argument which is compatible with many imitation versions
        # Adjust log_interval based on the total number of batches
        bc_trainer.train(n_batches=n_batches, log_interval=max(1, n_batches // 50)) # Log more frequently, e.g., 50 times per training run
    else:
        logging.warning("No training batches available. Skipping training.")


    # --- Save Policy ---
    if n_batches > 0: # Only attempt to save if training actually ran
        try:
            # The policy object is stored in bc_trainer.policy
            # Save it using the underlying SB3 policy save method
            # This saves an SB3-compatible policy, usually as a .zip file
            bc_trainer.policy.save(f"{POLICY_SAVE_PATH}.zip")
            logging.info(f"Card prediction policy saved successfully to {POLICY_SAVE_PATH}.zip")
        except Exception as e:
            logging.error(f"Error saving BC policy: {e}")
            traceback.print_exc()
    else:
        logging.warning("Skipping policy save because no training was performed.")


    logging.info("--- Card Policy Training Finished ---")

2025-04-20 12:59:31 - INFO - --- Starting Card Policy Training (BC) ---
2025-04-20 12:59:31 - INFO - Total valid data samples loaded for BC: 734009.
2025-04-20 12:59:31 - INFO - Splitting data for BC (keeping 85% for training)...
2025-04-20 12:59:32 - INFO - BC Training samples: 623907, Validation samples: 110102
2025-04-20 12:59:35 - INFO - Starting BC training for 10 epochs (~19498 batches per epoch) totaling 194980 batches...
  0%|          | 0/194980 [00:00<?, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.0022  |
|    entropy        | 2.2      |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 129      |
|    loss           | 2.19     |
|    neglogp        | 2.2      |
|    prob_true_act  | 0.111    |
|    samples_so_far | 32       |
--------------------------------


  0%|          | 511/194980 [00:06<40:48, 79.41batch/s] 


KeyboardInterrupt: 